In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 数据集

In [2]:
df = pd.read_csv('creditcard.csv')
df.head()
df['Class'].value_counts()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


Class
0    284315
1       492
Name: count, dtype: int64

# 平衡数据集

In [11]:
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(df.drop('Class', axis=1), df['Class'])


Class
0    284315
1    284315
Name: count, dtype: int64

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
cols = df.columns.tolist()
cols.remove('Class')
cols.remove('Time')
df[cols] = scaler.fit_transform(df[cols])


In [15]:
from sklearn.model_selection import train_test_split
X = df.drop('Class', axis=1)
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

In [16]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

# Isolation Forest
iso_forest = IsolationForest(contamination=0.01)
y_pred_iso = iso_forest.fit_predict(X_train)
y_pred_iso = [1 if x == 1 else 0 for x in y_pred_iso]  # 转换为1/0标签

# Local Outlier Factor
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.01)
y_pred_lof = lof.fit_predict(X_train)
y_pred_lof = [1 if x == 1 else 0 for x in y_pred_lof]  # 转换为1/0标签

In [17]:
from sklearn.metrics import classification_report, roc_auc_score

print("Isolation Forest 分类报告：")
print(classification_report(y_test, y_pred_iso))

print("LOF 分类报告：")
print(classification_report(y_test, y_pred_lof))

print("Isolation Forest AUC: ", roc_auc_score(y_test, y_pred_iso))
print("LOF AUC: ", roc_auc_score(y_test, y_pred_lof))

Isolation Forest 分类报告：


ValueError: Found input variables with inconsistent numbers of samples: [28481, 256326]